In [5]:
import ase
from ase.io import read, write, lammpsdata
from ase.build import make_supercell
import numpy as np

In [30]:
#atoms = lammpsdata.read_lammps_data("/Users/iananderson/Documents/Documents/gradschool/TaC_Screw_Dislocations/core_structure.lmp")
atoms = read("/Users/iananderson/Documents/Documents/gradschool/TaC_Screw_Dislocations/POSCAR.vasp", format='vasp')

In [33]:
transformation = np.array([
    [2,   0.0, 0.0],
    [12, 5, 4],
    [0.0,  0.0, 2.0]
])

# Apply the transformation
new_atoms = make_supercell(atoms, transformation)

# Save to file for LAMMPS
write("/Users/iananderson/Documents/Documents/gradschool/TaC_Screw_Dislocations/core_structure.lmp", atoms, format='lammps-data')